<a href="https://colab.research.google.com/github/DavidAlzateOcampo/DataScienceNetwork/blob/master/Tree_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Load packages
import pandas as pd
from sklearn.tree import DecisionTreeClassifier #The algorithm
from sklearn.model_selection import train_test_split #Train and test split function
from sklearn import metrics #To calculate model accuracy


In [0]:

#Loading the data
raw_dataset = pd.read_csv("/content/sample_data/bank-full.csv", header = 0, delimiter = ';')
raw_dataset.head()


In [0]:
raw_dataset.shape


+ (1) These 3 are all numerical. Modify the code so that you include categorical features of ‘marital’ and ‘education’ into the training and test datasets.

**David :** In the next chunk of code it was added the 2 new features marital and education.


In [0]:
#Lets do some feature selection 
feature_names = ['age', 'balance', 'duration','marital','education']
X = raw_dataset[feature_names] #Selected features
y = raw_dataset.y #Target feature (i.e., class label)


In [0]:

#The full dataset is randomly divided into the "train" and "test" sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1) #80% training and 20% test


+ (2) Try to train a decision tree classifier. What error do you get? What is the reason behind that?

**David :** The reason why the training fails is the way, fit method for DecisionTreeClassifier was implemented internally. In the documentation link in [here](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier.fit) it mentions that input X parameter must be an array-like or sparse matrix. Moreover, it mentiones that internally, it will be converted to dtype=np.float32.

Based on the documentation, the method does not expect a character feature, and that's the reason of the error. 

In [0]:
#Create Decision Tree classifer object
clf = DecisionTreeClassifier(criterion='entropy', max_depth = 3)
#Train a Decision Tree model
clf = clf.fit(X_train,y_train)
#Use the model to predict the response for our test dataset
y_pred = clf.predict(X_test)


+ (3) Find a way to pass the categorical variables to the Decision Tree algorithm in Scikit-Learn.

**David :** it was implemented the next code which verifies the type of the feature and for dtypes object, it appends the feature name as a categorical variable. Then, it obtains the numerical features. After that, it converts the categorical features to dummies columns, it means it create one column for each category of the features, and then it fills the samples with 1 or 0 in the new features. 

The reason why the training fails is the way, fit method for DecisionTreeClassifier was implemented internally. In the documentation link in [here](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier.fit) it mentions that input X parameter must be an array-like or sparse matrix. Moreover, it mentiones that internally, it will be converted to dtype=np.float32 .

Based on the documentation, the method does not expect a character feature, and that's the reason of the error. 

In [0]:
# Empty list to store columns with categorical data
categorical = []
for col, value in X.iteritems():
    if value.dtype == 'object':
        categorical.append(col)
        
# Store the numerical columns in a list numerical
numerical = X.columns.difference(categorical)

# Store the categorical data in a dataframe called attrition_cat
X_cat = X[categorical]

X_cat = pd.get_dummies(X_cat)

X_num = X[numerical]
# Concat the two dataframes together columnwise
X_final = []
X_final = pd.concat([X_num, X_cat], axis=1)
X_final.head()


In [0]:
#The full dataset is randomly divided into the "train" and "test" sets
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size = 0.2, random_state = 1) #80% training and 20% test

#Create Decision Tree classifer object
clf = DecisionTreeClassifier(criterion='entropy', max_depth = 3)
#Train a Decision Tree model
clf = clf.fit(X_train,y_train)
#Use the model to predict the response for our test dataset
y_pred = clf.predict(X_test)


 + (4) Did inclusion of new features improve the predictive accuracy of your model?
 
**David :** The accuracy remained the same. The reason is because the max_depth is 3 and it means new 2 features (marital, education) are not as important as the initial 3 features (age, balance, and duration). 

In [0]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))


**David:** I trained the model but changing the max_depth to 4. It slightly improved the accuracy by 0.03%. 

In [0]:
#Create Decision Tree classifer object
clf = DecisionTreeClassifier(criterion='entropy', max_depth = 4)
#Train a Decision Tree model
clf = clf.fit(X_train,y_train)
#Use the model to predict the response for our test dataset
y_pred = clf.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))


In [0]:
########## Visualizing the tree ##########
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus



In [0]:
print(feature_names)
feature_names = list(X_final.columns)
print(feature_names)

In [0]:
dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names = feature_names, class_names=['no','yes'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('bank_decision_tree.png')
Image(graph.create_png())
